In [1]:
############请勿修改##########
from math import sqrt
import matplotlib.pyplot as plt
from IPython.display import display, HTML


from cvxopt import matrix
import os, sys


from cvxopt.blas import dot
from cvxopt.solvers import qp
import numpy as np
import pandas as pd
def add_constraint(index, G, h, lb, ub):
    index -= 1
    if lb!=0:
        lrow = np.zeros(G.size[1])
        lrow[index] = -1
        G = matrix(np.vstack((G, lrow)))
        h = matrix(np.vstack((h, -lb)))
    if ub != 1:
        urow = np.zeros(G.size[1])
        urow[index] = 1
        G = matrix(np.vstack((G,urow)))
        h = matrix(np.vstack((h, ub)))
    return G,h
    
def add_return_goal(num, A, b):
    A = matrix(np.vstack((A, return_vector)))
    b = matrix(np.vstack((b, num)))
    return A, b

class HiddenPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._original_stdout

#############请勿修改#############

In [2]:
################设置参数##################

#资产数目
n = 5
    
#目标利率
return_goal = 0.05

#无风险利率（用来计算夏普率）
risk_free_rate = 0.02


In [3]:
###############读取EXCEL数据#################
#各资产波动率:读取同文件夹下excel文件 "均值方差模型.xlsx" 中的"波动率"表
sd = matrix(pd.read_excel("均值方差模型.xlsx", sheet_name="波动率",header=None).values)

#各资产预期收益率：读取同文件夹下excel文件 "均值方差模型.xlsx" 中的"预期收益率"表
return_vector = matrix(pd.read_excel("均值方差模型.xlsx",sheet_name="预期收益率",header=None).values, (1, n))

#各资产相关性系数：读取同文件夹下excel文件 "均值方差模型.xlsx" 中的"相关性"表
correlation = matrix(pd.read_excel("均值方差模型.xlsx",sheet_name="相关性",header=None).values)

In [4]:
##############请勿修改##############
cov = (np.array(sd) @ np.array(sd).transpose()) * np.array(correlation)

P = matrix(cov)

A = matrix(1.0, (1, n))

b = matrix(1.0)

G = matrix(0.0, (n,n))
G[::n+1] = -1.0


q = matrix(0.0, (n,1))

h = matrix(0.0, (n,1))
A,b = add_return_goal(return_goal, A, b)
##############请勿修改##############

In [5]:
############限制范围############

# 示例：G,h = add_constraint(4, G, h, 0.1, 0.5)
# 示例表示限制资产4的范围在10% - 50%之间

G,h = add_constraint(3, G, h, 0.1, 1)

G,h = add_constraint(4, G, h ,0, 0.3)

# G,h = add_constraint(5, G, h ,0, 0.05)

In [6]:
##########计算输出结果，请勿修改###########
model = qp(P, q, G, h, A, b)
print()
if model["status"] == "optimal":
    d = {"权重":[str(round(i * 100, 1))+"%" for i in model['x']],
         "目标收益率":[str(return_goal*100) + "%"] + [""] * (n-1),
        "组合风险率":[str(round(sqrt(model['primal objective'] *2)* 100,2) ) +'%'] + [""] * (n-1),
        "夏普比率":[round((return_goal - risk_free_rate )/sqrt(model['primal objective'] *2), 2)] + [""] * (n-1)}
    display(pd.DataFrame(d, index=["资产" + str(i+1) for i in range(n)]))
else:
    print("无解，请尝试调节收益率和限制范围后重试")
##############请勿修改##################

     pcost       dcost       gap    pres   dres
 0:  3.6085e-04 -1.2298e+00  1e+00  3e-16  3e+00
 1:  3.5976e-04 -1.2786e-02  1e-02  8e-17  4e-02
 2:  2.8915e-04 -4.3300e-04  7e-04  9e-17  2e-03
 3:  1.3209e-04  4.2221e-05  9e-05  2e-16  1e-18
 4:  1.0020e-04  9.3683e-05  7e-06  1e-16  3e-19
 5:  9.8888e-05  9.8770e-05  1e-07  6e-17  2e-19
 6:  9.8876e-05  9.8875e-05  1e-09  2e-17  4e-19
Optimal solution found.



,权重,目标收益率,组合风险率,夏普比率
资产1,7.5%,5.0%,1.41%,2.13
资产2,47.1%,,,
资产3,10.0%,,,
资产4,30.0%,,,
资产5,5.3%,,,


In [7]:
###########显示输入的数据###########

input_d = {"输入的波动率":np.array(sd).flatten(), "输入的收益率":np.array(return_vector).flatten()}
display(pd.DataFrame(input_d, index=["资产" + str(i+1) for i in range(n)]))
df2 = pd.DataFrame(np.array(correlation), index=["资产" + str(i+1) for i in range(n)], columns=["资产" + str(i+1) for i in range(n)])
df2.columns.name = "输入的相关性"
display(df2)
df3 = pd.DataFrame(cov, index=["资产" + str(i+1) for i in range(n)], columns=["资产" + str(i+1) for i in range(n)])
df3.columns.name = "计算的协方差"
display(df3)

,输入的波动率,输入的收益率
资产1,0.150,0.100
资产2,0.020,0.035
资产3,0.002,0.020
资产4,0.003,0.071
资产5,0.100,0.050


输入的相关性,资产1,资产2,资产3,资产4,资产5
资产1,1.00,-0.20,-0.14,-0.22,0.20
资产2,-0.20,1.00,0.08,0.48,-0.30
资产3,-0.14,0.08,1.00,0.06,-0.25
资产4,-0.22,0.48,0.06,1.00,-0.25
资产5,0.20,-0.30,-0.25,-0.25,1.00


计算的协方差,资产1,资产2,资产3,资产4,资产5
资产1,0.022500,-0.000600,-4.200000e-05,-9.900000e-05,0.003000
资产2,-0.000600,0.000400,3.200000e-06,2.880000e-05,-0.000600
资产3,-0.000042,0.000003,4.000000e-06,3.600000e-07,-0.000050
资产4,-0.000099,0.000029,3.600000e-07,9.000000e-06,-0.000075
资产5,0.003000,-0.000600,-5.000000e-05,-7.500000e-05,0.010000


In [8]:
#############设置画出图像的参数##########
# 收益率下限
lower_bound = 0.03

# 收益率上限
upper_bound = 0.13

In [9]:
############画图结果输出###########
x = []
y = []
i = lower_bound
result = True
with HiddenPrints():
    while i < upper_bound:
        return_goal = i
        A = matrix(1.0, (1, n))
        b = matrix(1.0)
        A,b = add_return_goal(return_goal, A, b)
        model = qp(P, q, G, h, A, b)
        if model["status"] != "optimal":
            result = False
            break
        x.append(i)
        y.append(sqrt(model['primal objective'] *2))
        i += 0.0001
if result == False:
    print("部分方程无解，请检测是否将上限设置的太高,或下限设置的太低")
else:
    x = np.array(x)
    y = np.array(y)
    plt.plot(y * 100, x * 100)
    plt.xlabel('volality: %')
    plt.ylabel('return: %')
    plt.show()


部分方程无解，请检测是否将上限设置的太高,或下限设置的太低
